In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [5]:
class Coupled_Oscillators:
    def __init__(self, N, m = 1.0, k_springs = 1.0, left_wall_k=0.0, right_wall_k=0.0):
          
        self.N=N
        self.M = self._build_mass_matrix(m)
        self.K = self._build_K_matrix(k_springs, left_wall_k, right_wall_k)

def _build_mass_matrix(self, m):

    if isinstance(m, (int, float)):  # todas as massas iguais
        masses = np.full(self.N, m)
    elif isinstance(m, (list, np.ndarray)) and len(m) == self.N:
        masses = np.array(m)
    else:
        raise ValueError("m deve ser um float ou um array com dimensão N")

        M = np.diag(masses)
        return M

def _build_K_matrix(self, k_springs, left_wall_k, right_wall_k):

    return K

def solve_coupled_system_linear(self, x0=None, v0=None, t_max=50, num_points=1000):

    return sol.t, sol.u, sol_euler.t, sol_euler_u
    